# Parallel computing in Julia
In this notebook it is shown how parallel computation can speed up complex computations

For this example, `QuadGK`, `Distributed`, `BenchmarkTools`, `SharedArrays` and `PyCall` are required, if not already installed please run the following code:

```julia
using Pkg
Pkg.add("QuadGK")
Pkg.add("Distributed")
Pkg.add("BenchmarkTools")
Pkg.add("SharedArrays")
Pkg.add("PyCall")

using Conda
Conda.install("scipy")

``` 



In [1]:
using Distributed
using BenchmarkTools

CPUcores=4

addprocs(CPUcores) # One should add n workers, where n is the number of available CPU cores
print(workers())

[2, 3, 4, 5]

I now need to load the library required for computing the integral (`QuadGK`) and the lib for `SharedArray`s.
Since every process need to be able to calculate integrals and operate on arrays, I load the libraies `@everywhere`

In [2]:
@everywhere using QuadGK
@everywhere using SharedArrays

Define the Euler $\Gamma$ function:
$$\Gamma(z)=\int_{0}^{\infty} x^{z-1}e^{-x}$$

In [3]:
@everywhere Γ(z)=quadgk(x->x^(z-1)*exp(-x),0,Inf32)

We shall now create 2 ```SharedArray```s. Such arrays can be accessed and modified by multiple processes simultaneously and in efficient fashion. They behave like regular arrays if no multiprocessing is required. 

In [4]:
npoints=10000
z = SharedArray(rand(range(1,stop=30, length=10000), npoints));
a = SharedArray(zeros(npoints)); # results array

Define a function used to fill `a` whith zeros

In [5]:
function reset_a!()
    @distributed for i=1:10000
        global a[i]=0
    end
end

reset_a! (generic function with 1 method)

I define two functions that will compute $\Gamma(z)$ at n points between 1:30 (up to 10000 points). The first one is parallelized, the second one not

<span style="color:red">Caution:</span> @synch is needed for the benchmark but usually it is not. 
Tasks can by run asynchronously or synchronously. <br>
A synchronous routine waits for all the tasks to finish before returning a results, while an asynchronous computation returns instantaneously a `Future` object, which will contains the results of the computation once it is done. <br>
Thus, in order to know the total compute time, it is preferable to run a synchronous task.

In [6]:
function test_me_distributed(n)
    @sync @distributed for i=1:n
        global a[i]=Γ(z[i])[1]
    end
end
test_me_distributed(3) #run it once so that it is already compiled

function test_me_distributed_async(n) #an asynchronous taks to show the results of an asynchronous computation
    @distributed for i=1:n
        global a[i]=Γ(z[i])[1]
    end
end
test_me_distributed_async(3)

function test_me(n)
    for i=1:n
        global a[i]=Γ(z[i])[1]
    end
end
test_me(3)

In [7]:
@time test_me_distributed_async(10000)
@time test_me_distributed(10000)
@time test_me(10000)

  0.000075 seconds (6 allocations: 928 bytes)
  2.685231 seconds (1.33 k allocations: 74.516 KiB)
  0.853919 seconds (28.22 M allocations: 440.353 MiB, 2.93% gc time)


As we can see, an async task requires virually no time, but the computation is still running when a future result is returned.

In [8]:
res1=0.
n=100
reset_a!()
for i=1:n
    tmp=@timed test_me_distributed(10000)
    global res1+=tmp[2]
end
print("elapsed time with parallelism: $(res1/n) seconds")

elapsed time with parallelism: 0.25808894303999996 seconds

In [9]:
res2=0.
reset_a!()
n=100
for i=1:n
    tmp=@timed test_me(10000)
    global res2+=tmp[2]
end
print("elapsed time single process: $(res2/n) seconds")


elapsed time single process: 0.8483086369600007 seconds

In [10]:
print("speed-up factor: $(res2/res1)")

speed-up factor: 3.2868848505010355

# Comparison with python

For reference, gamma(30) computed using quad in python, without optimization, requires ~410 $\mu s$, thus the whole test algorithm would require ~4s vs the julia 0.25s $\rightarrow$ 16x speedup. <br>
Now we will use the built-in scipy gamma function.

In [11]:
using PyCall

In [12]:
Γ_py=pyimport("scipy.special").gamma

PyObject <ufunc 'gamma'>

In [13]:
@time Γ_py(z)

  0.692003 seconds (1.42 M allocations: 72.234 MiB, 3.23% gc time)


10000-element Array{Float64,1}:
     1.2947359264256622e11
     3.2509308597513952e28
     0.924832458065631    
     4.1144441290405286e27
     6.500379348827447e9  
     4.2380409692267726e30
     4.905255534140806e24 
     1.2184097593626467e12
     1.410557008744064e11 
     1.3641668103462832e13
     1.0549529318242151e7 
 54945.54611094829        
     9.302839472887822e11 
     ⋮                    
     4.720226478792006e30 
     2.4536512621674158e27
     1.2957239287891263e20
     6.83193702121948e11  
     1.8941324203409977   
  5615.012472532945       
     6.881516491329566e25 
     7.232864198494877e29 
     1.4306561093697086   
     5.4347603515598996e25
   140.72170002594996     
     1.7244219195373504e9 

In [14]:
function test_me_py()
    a=SharedArray(Γ_py(z))
end
test_me_py();

In [15]:
@time test_me_py()

  0.006793 seconds (503 allocations: 96.094 KiB)


10000-element SharedArray{Float64,1}:
     1.2947359264256622e11
     3.2509308597513952e28
     0.924832458065631    
     4.1144441290405286e27
     6.500379348827447e9  
     4.2380409692267726e30
     4.905255534140806e24 
     1.2184097593626467e12
     1.410557008744064e11 
     1.3641668103462832e13
     1.0549529318242151e7 
 54945.54611094829        
     9.302839472887822e11 
     ⋮                    
     4.720226478792006e30 
     2.4536512621674158e27
     1.2957239287891263e20
     6.83193702121948e11  
     1.8941324203409977   
  5615.012472532945       
     6.881516491329566e25 
     7.232864198494877e29 
     1.4306561093697086   
     5.4347603515598996e25
   140.72170002594996     
     1.7244219195373504e9 

In [16]:
res3=0.
reset_a!()
n3=1000
for i=1:n
    tmp=@timed test_me_py()
    global res3+=tmp[2]
end
print("elapsed time python: $(res3/n3) seconds")

elapsed time python: 0.0008779030970000002 seconds

In [17]:
print("speed-up factor python-julia handwritten: $(res1/res3*(n3/n))")

speed-up factor python-julia handwritten: 293.9834065080191

Not so fast, unfortunately. Let's try the native Julia implementation of the gamma function (which is based on the GNU MPFR)

# Native gamma function parallelized

In [18]:
@everywhere using SpecialFunctions

In [19]:
function test_me_distributed_builtin(n)
    @sync @distributed for i=1:n
        global a[i]=gamma(z[i])
    end
end

function test_me_builtin(n)
    for i=1:n
        global a[i]=gamma(z[i])
    end
end

test_me_builtin (generic function with 1 method)

In [20]:
@time test_me_distributed_builtin(10)
@time test_me_builtin(10)

  0.387159 seconds (438.21 k allocations: 21.647 MiB, 2.90% gc time)
  0.012825 seconds (13.96 k allocations: 732.814 KiB)


In [21]:
res4=0.
reset_a!()
n4=1000
for i=1:n
    tmp=@timed test_me_distributed_builtin(n)
    global res4+=tmp[2]
end
print("elapsed time builtin gamma: $(res4/n4) seconds\n")

res5=0.
reset_a!()
n5=1000
for i=1:n
    tmp=@timed test_me_builtin(n)
    global res5+=tmp[2]
end
print("elapsed time builtin gamma: $(res5/n5) seconds")

elapsed time builtin gamma: 0.000198029904 seconds
elapsed time builtin gamma: 1.9630980000000007e-6 seconds

In [22]:
print("speed-up factor native-hand written: $(res1/res5*(n5/n)) \n")
print("speed-up factor 4-5: $(res4/res5) \n")
print("speed-up factor native-python: $(res3/res5*(n5/n3))")

speed-up factor native-hand written: 131470.2287099268 
speed-up factor 4-5: 100.87621911896397 
speed-up factor native-python: 447.2028890050318

As we observe, in this case there is no gain in parallelizing the gamma function. Since it is already really fast, the overhead generated by the parallelization tecnique is higher than the speed gain. 
Furthermore, an efficient algorithm is much better than the parallelization, leading to a performance increase of $10^5$. <br> 
As a side note, Julia is **100x faster than python** at computing the gamma function, when properly optimized